In [1]:
import pandas as pd
import math as mt
import numpy as np
import sys

# 1.数据预处理

In [2]:
df = pd.read_csv('.\dataset_diabetes\diabetic_data.csv')
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
# 删除同一病人的多个数据行
df = df.sort_values(by=['encounter_id', 'patient_nbr'], ascending=True)
df.drop_duplicates('patient_nbr', inplace=True)
df = df.reset_index(drop=True)
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
1,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
2,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
3,28236,89869032,AfricanAmerican,Female,[40-50),?,1,1,7,9,...,No,Steady,No,No,No,No,No,No,Yes,>30
4,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71513,443842016,183087545,Caucasian,Female,[70-80),?,1,1,7,9,...,No,Steady,No,No,No,No,No,Ch,Yes,>30
71514,443842022,188574944,Other,Female,[40-50),?,1,1,7,14,...,No,Up,No,No,No,No,No,Ch,Yes,>30
71515,443842070,140199494,Other,Female,[60-70),?,1,1,7,2,...,No,Steady,No,No,No,No,No,No,Yes,>30
71516,443842340,120975314,Caucasian,Female,[80-90),?,1,1,7,5,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [4]:
# 建立对应列处理字典
race_data = {"?":0,"AfricanAmerican":1,"Asian":2,"Caucasian":3,"Hispanic":4,"Other":5}
race_dict = pd.DataFrame(race_data, index=[0])
race_dict

,?,AfricanAmerican,Asian,Caucasian,Hispanic,Other
0,0,1,2,3,4,5


In [5]:
gender_data = {"Unknown/Invalid":0,"Male":1,"Female":2}
gender_dict = pd.DataFrame(gender_data, index=[0])
gender_dict

,Unknown/Invalid,Male,Female
0,0,1,2


In [6]:
age_data = {"[0-10)":0,"[10-20)":1,"[20-30)":2,"[30-40)":3,"[40-50)":4,"[50-60)":5,"[60-70)":6,"[70-80)":7,"[80-90)":8,"[90-100)":9}
age_dict = pd.DataFrame(age_data, index=[0])
age_dict

,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100)
0,0,1,2,3,4,5,6,7,8,9


In [7]:
weight_data = {"?":0,"[0-25)":1,"[25-50)":2,"[50-75)":3,"[75-100)":4,"[100-125)":5,"[125-150)":6,"[150-175)":7,"[175-200)":8,">200":9}
weight_dict = pd.DataFrame(weight_data, index=[0])
weight_dict

,?,[0-25),[25-50),[50-75),[75-100),[100-125),[125-150),[150-175),[175-200),>200
0,0,1,2,3,4,5,6,7,8,9


In [8]:
payer_code_unique = df["payer_code"].unique()
payer_code_data = {}
for i in range(0, len(payer_code_unique)):
    payer_code_data[payer_code_unique[i]] = i
payer_code_dict = pd.DataFrame(payer_code_data, index=[0])
payer_code_dict

,?,MC,MD,HM,UN,BC,SP,CP,SI,DM,CM,CH,PO,WC,OT,OG,MP,FR
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17


In [9]:
medical_specialty_unique = df["medical_specialty"].unique()
medical_specialty_data = {}
for i in range(0, len(medical_specialty_unique)):
    medical_specialty_data[medical_specialty_unique[i]] = i
medical_specialty_dict = pd.DataFrame(medical_specialty_data, index=[0])
medical_specialty_dict

,?,InternalMedicine,Family/GeneralPractice,Cardiology,Surgery-General,Orthopedics,Gastroenterology,Surgery-Cardiovascular/Thoracic,Nephrology,Orthopedics-Reconstructive,...,SportsMedicine,Speech,Hospitalist,OutreachServices,Cardiology-Pediatric,Perinatology,Neurophysiology,Endocrinology-Metabolism,DCPTEAM,Resident
0,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70


In [10]:
max_glu_serum_data = {"None":0,"Norm":1,">200":2,">300":3}
max_glu_serum_dict = pd.DataFrame(max_glu_serum_data, index=[0])
max_glu_serum_dict

,None,Norm,>200,>300
0,0,1,2,3


In [11]:
A1Cresult_data = {"None":0,"Norm":1,">7":2,">8":3}
A1Cresult_dict = pd.DataFrame(A1Cresult_data, index=[0])
A1Cresult_dict

,None,Norm,>7,>8
0,0,1,2,3


In [12]:
normal_data = {"No":0,"Steady":1,"Up":2,"Down":3}
normal_dict = pd.DataFrame(normal_data, index=[0])
normal_dict

,No,Steady,Up,Down
0,0,1,2,3


In [13]:
change_data = {"No":0,"Ch":1}
change_dict = pd.DataFrame(change_data, index=[0])
change_dict

,No,Ch
0,0,1


In [14]:
diabetesMed_data = {"No":0,"Yes":1}
diabetesMed_dict = pd.DataFrame(diabetesMed_data, index=[0])
diabetesMed_dict

,No,Yes
0,0,1


In [15]:
readmitted_data = {"NO":0,"<30":1,">30":2}
readmitted_dict = pd.DataFrame(readmitted_data, index=[0])
readmitted_dict

,NO,<30,>30
0,0,1,2


In [ ]:
for i in range(0, len(df)):
    df.loc[i, "race"] = race_dict[df["race"][i]][0]
    df.loc[i, "gender"] = gender_dict[df["gender"][i]][0]
    df.loc[i, "age"] = age_dict[df["age"][i]][0]
    df.loc[i, "weight"] = weight_dict[df["weight"][i]][0]
    df.loc[i, "payer_code"] = payer_code_dict[df["payer_code"][i]][0]
    df.loc[i, "medical_specialty"] =  medical_specialty_dict[df["medical_specialty"][i]][0]

    if (df["diag_1"][i] == '?') or ('V' in df["diag_1"][i]) or ('E' in df["diag_1"][i]):
        df.loc[i, "diag_1"] = 0
    if (df["diag_2"][i] == '?') or ('V' in df["diag_2"][i]) or ('E' in df["diag_2"][i]):
        df.loc[i, "diag_2"] = 0
    if (df["diag_3"][i] == '?') or ('V' in df["diag_3"][i]) or ('E' in df["diag_3"][i]):
        df.loc[i, "diag_3"] = 0

    df.loc[i, "max_glu_serum"] =  max_glu_serum_dict[df["max_glu_serum"][i]][0]
    df.loc[i, "A1Cresult"] =  A1Cresult_dict[df["A1Cresult"][i]][0]

    for j in range(24, 47):
        df.iloc[i, j] =  normal_dict[df.iloc[i, j]][0]

    df.loc[i, "change"] =  change_dict[df["change"][i]][0]
    df.loc[i, "diabetesMed"] =  diabetesMed_dict[df["diabetesMed"][i]][0]
    df.loc[i, "readmitted"] =  readmitted_dict[df["readmitted"][i]][0]

    print("当前处理进度：%d / %d" % (i, len(df)))

In [17]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,12522,48330783,3,2,8,0,2,1,4,13,...,0,1,0,0,0,0,0,1,1,0
1,15738,63555939,3,2,9,0,3,3,4,12,...,0,1,0,0,0,0,0,1,1,0
2,16680,42519267,3,1,4,0,1,1,7,1,...,0,1,0,0,0,0,0,1,1,0
3,28236,89869032,1,2,4,0,1,1,7,9,...,0,1,0,0,0,0,0,0,1,2
4,35754,82637451,3,1,5,0,2,1,2,3,...,0,1,0,0,0,0,0,0,1,2


In [18]:
output1 = '.\dataset_diabetes\diabetic_data2.csv'
df.to_csv(output1, sep=',', index=False, header=True)

In [19]:
df2 = pd.read_csv('.\dataset_diabetes\diabetic_data2.csv')

In [20]:
# 删除最后一列再入院数据
df2 = df2.drop(["readmitted"], axis=1)
# 删除前两列用于上述病人排序的序列数据
df2 = df2.drop(['encounter_id', 'patient_nbr'], axis=1)
# 删除取值唯一的数据列
df2 = df2.drop(['examide', 'citoglipton'], axis=1)

In [21]:
df2 = df2.astype('float')
df2.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,3.0,2.0,8.0,0.0,2.0,1.0,4.0,13.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,3.0,2.0,9.0,0.0,3.0,3.0,4.0,12.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3.0,1.0,4.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,1.0,2.0,4.0,0.0,1.0,1.0,7.0,9.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3.0,1.0,5.0,0.0,2.0,1.0,2.0,3.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
# 数据归一化
for column in df2:
    if df2.loc[:, column].min() == df2.loc[:, column].max():  #避免单值属性计算出bug
        df2.loc[:, column] = 0.0
    else:
        df2.loc[:, column] = \
            (df2.loc[:, column] - df2.loc[:, column].min()) / (df2.loc[:, column].max()- df2.loc[:, column].min())

df2.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,0.6,1.0,0.888889,0.0,0.142857,0.000000,0.125000,0.923077,0.0,0.000000,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.6,1.0,1.000000,0.0,0.285714,0.074074,0.125000,0.846154,0.0,0.014286,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.6,0.5,0.444444,0.0,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.2,1.0,0.444444,0.0,0.000000,0.000000,0.250000,0.615385,0.0,0.000000,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.6,0.5,0.555556,0.0,0.142857,0.000000,0.041667,0.153846,0.0,0.000000,...,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# 2.层次聚类

### 2.1 定义两点距离公式

In [23]:
def dis_between_two_point(point1, point2):
    '''
    这是一个计算不同的两个类簇中心点距离的函数
    :param point1:list
    :param point2:list
    :return:double
    '''
    dis = 0.0
    weishu = min(len(point1[0]), len(point2[0]))
    for i in range(1, weishu):
        dis += mt.pow(point1[0][i] - point2[0][i], 2)
    return mt.sqrt(dis)

### 2.2 定义层次聚类的数据结构——聚类树节点

In [24]:
class julei_node(object):
    def __init__(self, cn, cx = -1, ln = -1, rn = -1, dis = -1, ic = False):
        '''
        聚类树节点生成
        :param cn:list,左右子节点的中心节点的特征向量
        :param cx:np.julei_node,左右子节点的中心节点序号
        :param ln:int,左子节点序号
        :param rn:np.julei_node,右子节点序号
        :param ic:boolean,标识本节点（左右子节点的中心节点）是否已经参与过计算
        '''
        self.core = cn,
        self.core_node = cx,
        self.left_node = ln,
        self.right_node = rn,
        self.is_computed = ic

### 2.3 定义新聚类节点的计算方式

In [25]:
def core_node(node1, node2):
    '''
    理论上应该将两类簇的全部数据点取平均活得新的中心点/类簇点，考虑时间复杂度这里采用两类簇的中心点取平均作为新的中心点/类簇点
    :param node1: julei_node
    :param node2: julei_node
    :return: julei_node
    '''
    temp_point = []
    for i in range(0, len(node1.core[0])):
        temp_point.append((node1.core[0][i] + node2.core[0][i]) / 2.0)
    ans_core_node = julei_node(temp_point)
    ans_core_node.left_node = node1.core_node
    ans_core_node.right_node = node2.core_node
    return ans_core_node

### 2.4 定义层次聚类算法

#### 2.4.1 初始化节点池，假设每个样本点为一个簇类

In [26]:
node_pool = []
# for i in range(0, len(df2)):
# 由于层次聚类方法复杂度非常高，本数据集预处理后仍然有7000+个样本，所以为了演示下述代码效果，取前10条进行测试，实际如果时间允许，请您注释掉下面一行代码，并打开上面一行代码的注释
for i in range(0, 10):
    temp_node = julei_node(np.array(df2.iloc[i,:].values).tolist())
    temp_node.core_node = i
    node_pool.append(temp_node)

#### 2.4.2 定义相似矩阵计算函数

In [27]:
def similar_function(np):
    '''
    用于对节点池中未参与过计算的节点进行计算，并形成新的节点
    :param np: node_pool
    :return: True表明计算已结束
    '''
    doing = True
    xu = 1
    temp_node_pool = []
    while doing == True:
        temp_node_pool.clear()
        # 假设每个样本点为一个簇类 / 找到未计算的类簇核心
        for i in range(0, len(node_pool)):
            if(node_pool[i].is_computed == False):
                temp_node_pool.append(node_pool[i].core_node)
                # print(">获得待分类计算节点 %d" % (node_pool[i].core_node))
        print(temp_node_pool)
        if(len(temp_node_pool) > 1):
            # 计算相似矩阵
            # similar_matrix = np.zeros((len(temp_node_pool), len(temp_node_pool))),只是一种数据结构，没有必要存下来
            temp_min = sys.float_info.max
            temp_min_j = 0
            temp_min_k = 1
            for j in range(0, len(temp_node_pool)):
                # similar_matrix[j, j] = sys.float_info.max
                for k in range(j + 1, len(temp_node_pool)):
                    temp_dis = dis_between_two_point(node_pool[temp_node_pool[j]].core, node_pool[temp_node_pool[k]].core)
                    # similar_matrix[j, k] = temp_dis
                    # similar_matrix[k, j] = similar_matrix[j, k]
                    if temp_dis < temp_min:
                        temp_min = temp_dis
                        temp_min_j = temp_node_pool[j]
                        temp_min_k = temp_node_pool[k]
                        # print("%d+%d-%f" % (j,k,temp_min))
                # print(">>当前计算节点为 %d 和其他" % (node_pool[temp_node_pool[j]].core_node))
            new_node = core_node(node_pool[temp_min_j], node_pool[temp_min_k])
            node_pool[temp_min_j].is_computed = True
            node_pool[temp_min_k].is_computed = True
            new_node.core_node = len(node_pool)
            node_pool.append(new_node)
            print(">>>产生新类簇 %d，计算节点为 %d 和 %d" % (new_node.core_node, new_node.left_node, new_node.right_node))
        else:
            doing = False
        # print("当前为第 %d 次迭代计算，待计算节点数为 %d" % (xu, len(temp_node_pool)))
        xu += 1

In [28]:
similar_function(node_pool)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
>>>产生新类簇 10，计算节点为 0 和 1
[2, 3, 4, 5, 6, 7, 8, 9, 10]
>>>产生新类簇 11，计算节点为 2 和 8
[3, 4, 5, 6, 7, 9, 10, 11]
>>>产生新类簇 12，计算节点为 5 和 11
[3, 4, 6, 7, 9, 10, 12]
>>>产生新类簇 13，计算节点为 4 和 9
[3, 6, 7, 10, 12, 13]
>>>产生新类簇 14，计算节点为 3 和 7
[6, 10, 12, 13, 14]
>>>产生新类簇 15，计算节点为 13 和 14
[6, 10, 12, 15]
>>>产生新类簇 16，计算节点为 10 和 12
[6, 15, 16]
>>>产生新类簇 17，计算节点为 6 和 16
[15, 17]
>>>产生新类簇 18，计算节点为 15 和 17
[18]


In [29]:
def classify(node_pool, point, k):
    '''
    利用上述训练好的聚类树进行新样本的分类
    :param np: node_pool
    :param point: new point
    :param k: int,表明划分深度/种类数，如k=4表明整体有4类
    :return: 与哪一个样本最相似，当前返回聚类树中的分类路径
    '''
    ans_list = []
    temp_node = node_pool[len(node_pool) - 1]
    ans_list.append(temp_node.core_node)
    for i in range(0, k - 1):
        dis_left = dis_between_two_point(point, node_pool[temp_node.left_node].core)
        dis_right = dis_between_two_point(point, node_pool[temp_node.right_node].core)
        if dis_left < dis_right:
            temp_node = node_pool[temp_node.left_node]
        else:
            temp_node = node_pool[temp_node.right_node]
        ans_list.append(temp_node.core_node)
    return ans_list

In [30]:
test_point = np.array(df2.iloc[0,]).tolist()
test = (test_point, )

In [31]:
explain_list = classify(node_pool, test, 4)
for i in range(0, len(explain_list)):
    temp_str = " ".join(str(j) for j in node_pool[explain_list[i]].core[0])
    print("test_point 的 %d 级分类为 %s \n" % (i + 1, temp_str))

test_point 的 1 级分类为 0.47500000000000003 0.8125 0.625 0.0 0.0982142857142857 0.046296296296296294 0.16796875 0.4447115384615385 0.0 0.011607142857142856 0.3871660305343511 0.34895833333333337 0.193359375 0.0 0.005952380952380952 0.0 0.3896333833833834 0.4157107107107107 0.4034284284284284 0.4708333333333333 0.0 0.0 0.09375 0.16666666666666666 0.0 0.0 0.010416666666666666 0.0 0.03125 0.041666666666666664 0.0 0.0 0.020833333333333332 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.5 1.0 

test_point 的 2 级分类为 0.55 0.875 0.6111111111111112 0.0 0.08928571428571427 0.046296296296296294 0.19010416666666669 0.5240384615384616 0.0 0.01607142857142857 0.43463740458015265 0.11458333333333333 0.19296875 0.0 0.011904761904761904 0.0 0.3761886886886887 0.2751151151151151 0.23128128128128128 0.44166666666666665 0.0 0.0 0.1875 0.3333333333333333 0.0 0.0 0.020833333333333332 0.0 0.0625 0.08333333333333333 0.0 0.0 0.041666666666666664 0.0 0.0 0.0 0.0 0.6666666666666666 0.0 0.0 0.0 0.0 0.0 1.0 1.0 

test_point 

### 2.5 效果评估

#### 2.5.1 CPCC系数计算

In [32]:
# 这里只实现CPCC，其他评估函数均可直接调用sklearn包实现
def CPCC(test_point, ans_point):
    '''
    共性分类相关系数，用于判别层次聚类的聚类结果是否和输入强相关，取值越接近1越好
    :param test_point: 待分类样本
    :param ans_point: 待分类样本在聚类树中最可能存在的位置/最相似样本
    :return: 相关系数
    '''
    ans = 0.0
    fenmu1 = 0.0
    for i in range(0,len(test_point[0])):
        fenmu1 += pow(test_point[0][i], 2)
    fenmu1 = mt.sqrt(fenmu1)
    fenmu2 = 0.0
    for i in range(0,len(ans_point[0])):
        fenmu2 += pow(ans_point[0][i], 2)
    fenmu2 = mt.sqrt(fenmu2)
    fenzi = 0.0
    for i in range(0,len(ans_point[0])):
        fenzi += test_point[0][i] * ans_point[0][i]
    ans = fenzi / (fenmu1 * fenmu2)
    return ans

In [33]:
CPCC(test, node_pool[explain_list[len(explain_list) - 1]].core)

0.9873232630653909